In [1]:
import quandl
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r'C:\Users\User\Documents\AMFI_metadata.csv')
#Importing list of codes,names of mutual funds, starting dates and end dates from the csv file stored locally

In [3]:
temp=df["code"].to_numpy()#Storing all codes as numpy array
t=np.char.mod("%d",temp) #Temporary numpy that stores all codes in numpy format
codes=np.char.add("AMFI/",t)#Final numpy containing codes of M.F.s in required format to request historical data for that 
                            # particular M.F. from Quandl

s_date=df["from_date"].to_numpy()#Storing start dates

e_date=df["to_date"].to_numpy()#Storing end dates

names=df["name"].to_numpy()#Storing names of all the M.F.s corresponding to the codes

# e_date also contains some null dates like MF's that have already expired please remove those MF's in order to save time

In [ ]:
quandl.ApiConfig.api_key = 'WKxt2giAPZ261LFGJ6m1'
for i in range(len(codes)):
    
    data=quandl.get(codes[i],start_date=s_date[i],end_date=e_date[i])#Importing data for each code from Quandl
    df1=pd.DataFrame(data)#Storing the imported data in dataframe
    
    df1.insert(0,"Name of Fund",names[i])#Creating a column to store name of fund
    df1.insert(2,"Category:",Classification(names[i]))#Creating a column that stores the the caegory under which the M.F. falls
    
    df1.to_csv('MutualFundsData.csv',mode='a',header=True)#Creating and appending all the data stored in dataframe into this csv
    print("No.of MFs added:",i)

In [ ]:
#List of keys for each type of M.F. used in the classification function
equityKeys=['equity','contra','dividend yield','elss','focused','large and mid cap','large cap','mid cap','multi cap','small cap','value','unit linked insurance scheme','growth','dividend plan','direct dividend']
debtKeys=['debt','banking and psu','corporate bond','credit risk','dynamic','floater','gilt','gilt 10 year','liquid','long duration','medium duration','medium to long duration','money market','overnight','short duration','ultra short duration']
hybridKeys=['hybrid','aggressive','arbitrage','balanced advantage','conservative','multi asset allocation','equity savings']
solutionOrientedKeys=['children','retirement']
otherKeys=['etf','fof domestic','fof overseas','index']


In [ ]:
#This function is used to accept name of M.F. and split it up into a list containing words of that name excluding special characters

def split(text):
    temp=text.replace("-"," ")
    temp=temp.replace("("," ")
    temp=temp.replace(")"," ")
    temp=temp.replace(","," ")
    temp=temp.replace("'"," ")
    temp=temp.split(" ")
    return temp

In [ ]:
#This function is used to parse through name and search for keywords
def Classification(text):
    
    keys=np.array([" Equity "," Debt "," Hybrid "," Solution Oriented "," Others "])
    
    temp=split(text)# Name of M.F. stored as list of words
    
    count=np.zeros((1,3))#Used to store 3 values, first is a temporary value that evaluates frequency of repetition of a key
                         #second value stores position of keyword in keywords numpy closest to words in list
                         #Third value stores which type of M.F. classification it falls under like equity or debt.
        
    
    temp1=0#Temporary variable used to store frequency by which a keyword repeats in name of M.F.
    for j in range(len(equityKeys)):
        
        for i in temp:
            if((i.lower()) in equityKeys[j]):#Comparing each word in name with each word in keyword arrays
                temp1=temp1+1                #Incrementing frequency count
                count[0][1]=j               #Storing the position ofthe key word with positive hit
        
        temp2=" "
        if(equityKeys[int(count[0][1])] in temp2.join(temp)):#Checkin if a keyword is directly found in name as sometimes due
            return keys[0]+" "+equityKeys[int(count[0][1])]  #to overlapping words in keywords most accurate value is missed
        
        if(temp1>count[0][0]):
            count[0][0]=temp1#Stores frequency of matches
            count[0][1]=j#Position in keyword list
            count[0][2]=0#To which list this keyword belongs
    
    
    temp1=0        
    for j in range(len(debtKeys)):
        for i in temp:
            if((i.lower()) in debtKeys[j]):#Comparing each word in name with each word in keyword arrays
                temp1=temp1+1                #Incrementing frequency count
                count[0][1]=j               #Storing the position ofthe key word with positive hit
                
        temp2=" "
        if(debtKeys[int(count[0][1])] in temp2.join(temp)):
            return keys[1]+" "+debtKeys[int(count[0][1])]
        
        if(temp1>count[0][0]):
            count[0][0]=temp1#Stores frequency of matches
            count[0][1]=j#Position in keyword list
            count[0][2]=1#To which list this keyword belongs
    
    
    temp1=0        
    for j in range(len(hybridKeys)):
        
        for i in temp:
            if((i.lower()) in hybridKeys[j]):#Comparing each word in name with each word in keyword arrays
                temp1=temp1+1                #Incrementing frequency count
                count[0][1]=j               #Storing the position ofthe key word with positive hit
                
        temp2=" "
        if(hybridKeys[int(count[0][1])] in temp2.join(temp)):
            return keys[2]+" "+hybridKeys[int(count[0][1])]
        
        if(temp1>count[0][0]):
            count[0][0]=temp1#Stores frequency of matches
            count[0][1]=j#Position in keyword list
            count[0][2]=2#To which list this keyword belongs
    
    
    temp1=0        
    for j in range(len(solutionOrientedKeys)):
        
        for i in temp:
            if((i.lower()) in solutionOrientedKeys[j]):#Comparing each word in name with each word in keyword arrays
                temp1=temp1+1                #Incrementing frequency count
                count[0][1]=j               #Storing the position ofthe key word with positive hit
                
        temp2=" "
        try:
            if(solutionOrientedKeys[int(count[0][1])] in temp2.join(temp)):
                return keys[3]+" "+solutionOrientedKeys[int(count[0][1])]
            
        except IndexError:
            return "Unable to process"#An error that sometimes arises if name of M.F. has almost no hits
                                      
        
        if(temp1>count[0][0]):
            count[0][0]=temp1#Stores frequency of matches
            count[0][1]=j#Position in keyword list
            count[0][2]=3#To which list this keyword belongs
    
    
    temp1=0        
    for j in range(len(otherKeys)):
        
        for i in temp:
            if((i.lower()) in otherKeys[j]):#Comparing each word in name with each word in keyword arrays
                temp1=temp1+1                #Incrementing frequency count
                count[0][1]=j               #Storing the position ofthe key word with positive hit
                
        temp2=" "
        if(otherKeys[int(count[0][1])] in temp2.join(temp)):
            return keys[4]+" "+otherKeys[int(count[0][1])]
        
        if(temp1>count[0][0]):
            count[0][0]=temp1#Stores frequency of matches
            count[0][1]=j#Position in keyword list
            count[0][2]=4#To which list this keyword belongs
            
    
    
    #BElow based on the major classification and major hit in keywords are returned as a single word and stored as the category
    # of that particular Mutual Fund
    
    if (count[0][2]==0):
            return (keys[int(count[0][2])])+" "+(equityKeys[int(count[0][1])])
    elif (count[0][2]==1):
            return (keys[int(count[0][2])])+" "+(debtKeys[int(count[0][1])])
    elif (count[0][2]==2):
            return (keys[int(count[0][2])])+" "+(hybridKeys[int(count[0][1])])
    elif (count[0][2]==3):
            return (keys[int(count[0][2])])+" "+(solutionOrientedKeys[int(count[0][1])])
    elif (count[0][2]==4):
            return keys[int(count[0][2])]+" "+otherKeys[int(count[0][1])]

# DISCLAIMER! PLEASE MAKE SURE THERE ARE NO "UNABLE TO PROCESS" HEADINGS IN CATEGORY AS THESE ARE NAMES WITH NO HITS IN KEYWORDS